In [ ]:
import pandas as pd
import numpy as np
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
import os
import warnings

# Configuración inicial para manejo de advertencias y asegurarnos de que las gráficas se muestren
warnings.filterwarnings('ignore', category=RuntimeWarning, module='mne')
mne.set_log_level('WARNING') # Ajustado a WARNING para ver errores críticos, pero no el spam de INFO

# --- [ SECCIÓN 1: PARÁMETROS Y CARGA DE DATOS DE OPENBCI ] ---

# 1. Definición de Parámetros
SFREQ = 250  # Frecuencia de muestreo (Sample Rate = 250 Hz, según tu archivo)
N_CHANNELS = 8  # Número de canales EEG (8 EXG)

# ******************************************************************************
# * RUTA FINAL: Usando la ruta absoluta que proporcionaste.
# ******************************************************************************
file_path = '/Users/lucianatarazona/Downloads/OpenBCI-RAW-2025-09-17_12-45-51 (1).txt' 

# ----------------- DIAGNÓSTICO DE RUTA -----------------
# El script ahora verifica la ruta absoluta:
print(f"Buscando el archivo en la ruta: {file_path}")
# -------------------------------------------------------

# Verificar existencia del archivo
if not os.path.exists(file_path):
    print(f"================================================================")
    print(f"ERROR FATAL: No se encontró el archivo '{file_path}'.")
    print(f"Verifica la ruta en la línea 17. Debe ser EXACTA.")
    print(f"================================================================")
    exit()

print(f"--- 1. Cargando datos desde: {file_path} ---")

# 2. Cargar datos usando Pandas
# CORRECCIÓN CLAVE: Usamos tabulación ('\t') como delimitador
try:
    data_df = pd.read_csv(
        file_path,
        sep='\t', # <-- CORRECCIÓN FINAL A TABULACIÓN
        skiprows=5,
        index_col=0 # Usar 'Sample Index' como índice
    )
except Exception as e:
    print(f"================================================================")
    print(f"ERROR: Falló al leer el contenido del archivo con el formato esperado. {e}")
    print(f"================================================================")
    exit()

# 3. Seleccionar solo los datos de EEG (columnas EXG Channel 0 a 7)
# Se usa ' EXG Channel X' con el espacio, ya que así aparece en el encabezado de tu archivo.
eeg_data_columns = [f' EXG Channel {i}' for i in range(N_CHANNELS)]

# Verificar si las columnas de EEG existen antes de usarlas
missing_cols = [col for col in eeg_data_columns if col not in data_df.columns]
if missing_cols:
    print(f"================================================================")
    print(f"ERROR: No se encontraron las columnas de EEG. El formato del archivo es INESPERADO.")
    print(f"Columnas que no se encontraron: {missing_cols}")
    print(f"Intenta abrir el archivo y verificar el espaciado exacto en el encabezado.")
    print(f"================================================================")
    exit()

# Extraer y transponer datos (MNE espera canales x tiempo)
# Convertir de microvoltios (uV) a voltios (V) para MNE.
eeg_data_microvolts = data_df[eeg_data_columns].values.T
eeg_data_volts = eeg_data_microvolts / 1000000.0

# 4. Crear el objeto MNE Info y RawArray
# Nombres de canales típicos para Ultracortex 8 canales (basado en el montaje 10-20 estándar)
ch_names = ['Fz', 'Cz', 'Pz', 'Oz', 'C3', 'C4', 'T7', 'T8'] 
ch_types = ['eeg'] * N_CHANNELS
info = mne.create_info(ch_names=ch_names, sfreq=SFREQ, ch_types=ch_types)

# Crear el objeto Raw de MNE
raw = mne.io.RawArray(eeg_data_volts, info)
raw.set_montage('standard_1020') # Asignar montaje para visualización

print(f"Objeto MNE Raw creado: {raw.info['nchan']} canales, {raw.n_times / raw.info['sfreq']:.2f} segundos.")

# Visualizar datos crudos (se abre una ventana, cierra para continuar)
raw.plot(duration=10, n_channels=N_CHANNELS, title='EEG Crudo (Raw Data) - Antes de ICA')


# --- [ SECCIÓN 2: PREPROCESAMIENTO PARA ICA ] ---

print("--- 2. Aplicando Filtrado Pasa-Alto (1.0 Hz) ---")

# 5. Filtrado Pasa-Alto (High-pass)
# Es esencial para eliminar derivas lentas antes de ICA.
raw.load_data() 
raw.filter(l_freq=1.0, h_freq=None, fir_design='firwin')

# --- [ SECCIÓN 3: APLICACIÓN DEL ALGORITMO ICA ] ---

print("--- 3. Ejecutando ICA (Independent Component Analysis) ---")

# 6. Run ICA
# Se usa n_components=7 (N_CHANNELS - 1) para 8 canales.
ica = ICA(n_components=7, method="picard", max_iter="auto", random_state=97)
ica.fit(raw)

print("ICA completado. Abriendo ventanas para inspección manual de componentes...")

# --- [ SECCIÓN 4: VISUALIZACIÓN E INSPECCIÓN MANUAL ] ---

# 7. Inspección de Componentes
print("\n--- PASO CLAVE: Inspección de Componentes ICA ---")
print("Busca componentes con:")
print("1. Alta frecuencia, forma de 'ruido' o picos rápidos en el curso temporal.")
print("2. Pendiente positiva en el espectro de potencia (Power Spectrum) a partir de 20-30 Hz.")
print("3. Topografía periférica (mapa topográfico concentrado en bordes/sienes).")

# Visualización Clave (se abren 2 ventanas)
# 
# Primera visualización: Cursos temporales de los componentes
ica.plot_sources(raw, title='Fuentes ICA (Time Courses) - Cierra para ver Propiedades')

# Segunda visualización: Propiedades (Topomap, Espectro, Curso Temporal).
# Esta es la ventana CLAVE para identificar el artefacto muscular.
ica.plot_properties(raw, picks=range(ica.n_components_), log_scale=True, 
                    title='Propiedades ICA - ¡Busca el Artefacto Muscular!')


# --- [ SECCIÓN 5: REMOCIÓN DEL ARTEFACTO (AJUSTE MANUAL REQUERIDO) ] ---

# 8. Ejemplo de Remoción
#
# *PASO MANUAL*
# 1. Mira las gráficas del paso 7.
# 2. Identifica el índice (0 a 6) de los componentes que representan el artefacto muscular.
# 3. Reemplaza los índices en la lista `muscle_idx_found`.

# ******************************************************************************
# * REEMPLAZA ESTOS ÍNDICES CON LOS QUE ENCUENTRES EN LA INSPECCIÓN VISUAL     *
# * (Ejemplo: muscle_idx_found = [4, 6])                                       *
# ******************************************************************************
muscle_idx_found = []  # <--- INICIALMENTE VACÍO. 

print("\n--- 4. Aplicación de ICA (Exclusión de Artefactos) ---")

# Copiar los datos raw para aplicar ICA sin modificar el original
raw_clean = raw.copy() 

# Aplicar la exclusión
if muscle_idx_found:
    ica.apply(raw_clean, exclude=muscle_idx_found)
    print(f"Componentes excluidos: {muscle_idx_found}. Señal limpia creada.")
    
    # Visualizar la señal limpia después de la remoción
    raw_clean.plot(duration=10, n_channels=N_CHANNELS, 
                   title=f'EEG Limpio después de ICA (Excluidos: {muscle_idx_found})')
else:
    print("Ningún componente excluido (muscle_idx_found está vacío). Por favor, revisa las gráficas y actualiza la lista en el código.")

plt.show() # Mostrar todas las gráficas pendientes
print("\nProceso de filtrado ICA completado. Revisa las gráficas para la inspección final.")

Buscando el archivo en la ruta: /Users/lucianatarazona/Downloads/OpenBCI-RAW-2025-09-17_12-45-51 (1).txt
--- 1. Cargando datos desde: /Users/lucianatarazona/Downloads/OpenBCI-RAW-2025-09-17_12-45-51 (1).txt ---
ERROR: No se encontraron las columnas de EEG. El formato del archivo es INESPERADO.
Columnas que no se encontraron: [' EXG Channel 0', ' EXG Channel 1', ' EXG Channel 2', ' EXG Channel 3', ' EXG Channel 4', ' EXG Channel 5', ' EXG Channel 6', ' EXG Channel 7']
Intenta abrir el archivo y verificar el espaciado exacto en el encabezado.


KeyError: "None of [Index([' EXG Channel 0', ' EXG Channel 1', ' EXG Channel 2', ' EXG Channel 3',\n       ' EXG Channel 4', ' EXG Channel 5', ' EXG Channel 6', ' EXG Channel 7'],\n      dtype='object')] are in the [columns]"

: 